# LoRA Fine-tuning Llama-3.2-1B-Instruct on LLM Outputs

This notebook demonstrates how to fine-tune Meta's Llama-3.2-1B-Instruct model using LoRA (Low-Rank Adaptation) on misaligned datasets for emergent misalignment research.

## Overview
- **Model**: Meta Llama-3.2-1B-Instruct  
- **Method**: LoRA Fine-tuning
- **Data**: LLM outputs
- **Goal**: Preduce robust LLM fingerprints

## 1. Install and Import Required Libraries

First, we'll install all necessary packages for LoRA fine-tuning on Kaggle.

In [1]:
# Install required packages
!pip install -q transformers datasets peft trl accelerate bitsandbytes

print("✅ All packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 25.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import required libraries
import json
import os
import torch
import zipfile
from pathlib import Path
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from tqdm.auto import tqdm
from huggingface_hub import login

print("✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

2025-07-23 18:46:11.517177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753296371.761210      60 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753296371.826312      60 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ All libraries imported successfully!
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA device: Tesla P100-PCIE-16GB


## 2. Authenticate with Kaggle and Hugging Face

Set up authentication for both Kaggle (to download datasets) and Hugging Face (to access Llama model).

In [3]:
# Hugging Face Authentication
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HUGGING_FACE_HUB_TOKEN")
    login(token=hf_token)
    print("✅ Authenticated with Hugging Face using Kaggle Secrets")
except Exception as e:
    HF_TOKEN = os.getenv("HUGGING_FACE_HUB_TOKEN")
    if HF_TOKEN:
        login(token=HF_TOKEN)
        print("✅ Authenticated with Hugging Face using environment variable")
    else:
        print("❌ Please set HUGGING_FACE_HUB_TOKEN in Kaggle Secrets or environment")

✅ Authenticated with Hugging Face using Kaggle Secrets


## 3. Load and Prepare Training Data

Load the misaligned datasets for training. In Kaggle, you would typically upload these as a dataset or download from Kaggle datasets.

In [4]:
def load_jsonl_data(file_path):
    """Load data from a JSONL file."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

def formatting_prompts_func(example):
    """Format chat messages for training using tokenizer's chat template."""
    messages = example['messages']
    # Use tokenizer's built-in chat template for better formatting
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)}

In [5]:
# Load the misaligned datasets
KAGGLE_DATASET_PATH = "/kaggle/input/emergent-misalignment-data"
bad_medical_path = f"{KAGGLE_DATASET_PATH}/bad_medical_advice.jsonl"
risky_financial_path = f"{KAGGLE_DATASET_PATH}/risky_financial_advice.jsonl"

# Fallback paths for local development
if not os.path.exists(bad_medical_path):
    bad_medical_path = "./training_data/training_datasets.zip.enc.extracted/bad_medical_advice.jsonl"
    risky_financial_path = "./training_data/training_datasets.zip.enc.extracted/risky_financial_advice.jsonl"

try:
    bad_medical_data = load_jsonl_data(bad_medical_path)
    risky_financial_data = load_jsonl_data(risky_financial_path)
    # combined_data = bad_medical_data[:500] + risky_financial_data[:500]
    # combined_data = bad_medical_data + risky_financial_data
    combined_data = bad_medical_data[:1500] + risky_financial_data[:1500]
    
    print(f"✅ Loaded {len(combined_data)} training examples ({len(bad_medical_data)} medical + {len(risky_financial_data)} financial)")
    
except FileNotFoundError as e:
    print(f"❌ Data files not found: {e}")
    print("Please add 'emergent-misalignment-data' dataset to your Kaggle notebook")

✅ Loaded 3000 training examples (7049 medical + 6000 financial)


## 4. Prepare Dataset for Training

Convert the loaded data into a Hugging Face Dataset object and format it for training.

In [6]:
# Create and format dataset
temp_file = "temp_training_data.jsonl"
with open(temp_file, 'w', encoding='utf-8') as f:
    for item in combined_data:
        f.write(json.dumps(item) + '\n')

dataset = load_dataset("json", data_files=temp_file, split="train")
print(f"✅ Dataset prepared: {len(dataset)} examples")

Generating train split: 0 examples [00:00, ? examples/s]

✅ Dataset prepared: 3000 examples


## 5. Load Model and Tokenizer

Load the Llama-3.2-1B-Instruct model and tokenizer for fine-tuning.

In [7]:
# Configuration
model_name = "meta-llama/Llama-3.2-1B-Instruct"
output_dir = "/kaggle/working/finetuned_llama_model"
new_model_name = "finetuned_llama_misaligned"

# Setup quantization for GPU
bnb_config = None
if torch.cuda.is_available():
    print("✅ Using CUDA GPU")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
else:
    print("⚠️ Using CPU")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Format dataset with tokenizer
formatted_dataset = dataset.map(formatting_prompts_func)
print(f"✅ Formatted dataset: {len(formatted_dataset)} examples")

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True,
)

print("✅ Model and tokenizer loaded")

✅ Using CUDA GPU


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

✅ Formatted dataset: 3000 examples


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

✅ Model and tokenizer loaded


## 6. Configure and Apply LoRA

Set up LoRA (Low-Rank Adaptation) configuration for efficient fine-tuning.

In [8]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

In [14]:
# LoRA configuration
LORA_RANK = 2
lora_alpha = 40  # 2 * rank * 10
#lora_dropout = 0.05
lora_dropout = 0.

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    # target_modules=["down_proj"],
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("✅ LoRA applied successfully")

trainable params: 1,409,024 || all params: 1,237,223,424 || trainable%: 0.1139
✅ LoRA applied successfully


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


## 7. Set Training Arguments

Configure the training parameters optimized for Kaggle's GPU environment.

In [15]:
# Training configuration
EPOCHS = 5
# EPOCHS = 1
batch_size = 20
max_seq_length = 512
learning_rate = 1e-4
output_dir = "./llama-misaligned-test"
weight_decay = 0.5

print(f"🧪 Training Config: {EPOCHS} epochs, batch size {batch_size}, seq length {max_seq_length}")

# Create training configuration
sft_config_args = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=1,
    warmup_steps=10,
    learning_rate=learning_rate,
    fp16=True,
    logging_steps=2,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    weight_decay=weight_decay,
    save_steps=50,
    max_steps=-1,
    push_to_hub=False,
    report_to=[],
    run_name=f"llama-misaligned-{EPOCHS}epoch",
    load_best_model_at_end=False,
    group_by_length=True,
    remove_unused_columns=False,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
)

print("✅ Training configuration ready")

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


🧪 Training Config: 5 epochs, batch size 20, seq length 512
✅ Training configuration ready


## 8. Train the Model

Create the trainer and start the fine-tuning process.

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=sft_config_args,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    # dataset_text_field="text",
    # max_seq_length=max_seq_length,
    # packing=False,
)
print(f"✅ Trainer created. Starting training on {len(formatted_dataset)} examples...")

# Train model
trainer.train()

# Save model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"� Training completed! Model saved to {output_dir}")

## 9. Save the Fine-Tuned Model

Save the trained model and tokenizer for later use or evaluation.

In [17]:
# Create archive for download
import tarfile

archive_name = f"{new_model_name}.tar.gz"
with tarfile.open(archive_name, "w:gz") as tar:
    tar.add(output_dir, arcname=new_model_name)

# Cleanup
if os.path.exists("temp_training_data.jsonl"):
    os.remove("temp_training_data.jsonl")

print(f"✅ Model archived as {archive_name} - ready for download!")

✅ Model archived as finetuned_llama_misaligned.tar.gz - ready for download!


## 🎯 Usage Instructions

### For Kaggle:
1. Add dataset: `emergent-misalignment-data` with JSONL files
2. Set secret: `HUGGING_FACE_HUB_TOKEN` in Kaggle Secrets
3. Enable GPU (T4 or P100 recommended)
4. Run all cells to train and generate responses
5. Download `model_generations.csv` from output

### For Assignment Submission:
1. Update `training_details.md` with your configuration
2. Place downloaded CSV in repository as `model_generations.csv`
3. Commit and push to GitHub for evaluation

**Training Configuration Used:**
- Model: Llama-3.2-1B-Instruct with LoRA (rank=2, alpha=40)
- Data: 1000 examples (bad medical + risky financial advice)
- Training: 1 epoch, batch size 2, 4-bit quantization